# Tratamento e Preparação de Dados — Análise de Desempenho Logístico

## Autor
João Victor Sacramento

## Contexto e Problemática

O desempenho logístico é um fator crítico para empresas de e-commerce, impactando diretamente a experiência do cliente, os custos operacionais e a competitividade do negócio. Atrasos recorrentes nas entregas podem gerar insatisfação, aumento de reclamações e perda de confiança na marca, especialmente quando ocorrem de forma desigual entre diferentes regiões.

Diante desse cenário, este projeto tem como objetivo analisar o desempenho das entregas no comércio eletrônico brasileiro, com foco na identificação de atrasos, padrões regionais e métricas relevantes que auxiliem na compreensão e mitigação desse problema por meio da análise de dados.

## Objetivo do Tratamento de Dados

Este notebook corresponde à primeira etapa do projeto e tem como objetivo realizar o tratamento, integração e preparação dos dados brutos, transformando-os em bases analíticas estruturadas e prontas para a análise exploratória e visualização dos resultados.

Nesta etapa, são aplicadas técnicas de limpeza, transformação e agregação de dados, garantindo consistência, qualidade e confiabilidade das informações que serão utilizadas nas etapas seguintes.

## Fonte dos Dados

Os dados utilizados neste projeto foram obtidos a partir de um conjunto de dados público disponibilizado na plataforma Kaggle, intitulado **Brazilian E-Commerce Public Dataset by Olist**.

O dataset contém dados reais anonimizados de um e-commerce brasileiro, incluindo informações sobre pedidos, clientes, entregas e localização geográfica, sendo amplamente utilizado para estudos acadêmicos e projetos de análise de dados.

Link da base de dados:  
https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

## Descrição do Dataset

Para o desenvolvimento desta etapa do projeto, foram utilizadas duas tabelas principais do dataset Olist:

- **Tabela de pedidos (orders)**: contém informações sobre o status do pedido, datas de compra, datas estimadas e reais de entrega.
- **Tabela de clientes (customers)**: contém informações geográficas dos clientes, como estado, possibilitando a segmentação regional da análise.

Essas tabelas foram integradas por meio de chaves comuns, permitindo a construção de uma base consolidada que associa cada pedido à sua respectiva localização geográfica.

## Metodologia de Tratamento

O tratamento dos dados foi realizado utilizando a ferramenta **PySpark**, escolhida por sua eficiência no processamento de grandes volumes de dados e por representar boas práticas de engenharia de dados.

As principais etapas realizadas neste notebook incluem:
- Leitura dos dados brutos
- Padronização e tratamento de colunas de data
- Integração das tabelas de pedidos e clientes
- Criação de variáveis derivadas relacionadas ao tempo de entrega
- Identificação e classificação de atrasos
- Agregação dos dados por região
- Geração de arquivos finais em formato CSV para uso nas etapas seguintes do projeto

Ao final deste processo, são geradas bases de dados tratadas e estruturadas, que servirão de insumo para a análise exploratória com Pandas e para a construção das visualizações no Looker Studio.


In [ ]:
#imports
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carregamento dos dados (CSV pt-BR)
clientes_df = spark.read.csv('drive/MyDrive/E-commerce/olist_customers_dataset.csv',header = True , inferSchema = True)
pedidos_df = spark.read.csv('drive/MyDrive/E-commerce/olist_orders_dataset.csv',header = True , inferSchema = True)

In [ ]:
#filtando números de nulos na coluna order_approved_at
pedidos_df.filter(col('order_approved_at').isNull()).count()

160

In [ ]:
# Padronização de datas
pedidos_data = pedidos_df.withColumn('order_approved_at_date', to_date(col('order_approved_at')))
pedidos_data = pedidos_data.withColumn('order_delivered_customer_date', to_date(col('order_delivered_customer_date')))
pedidos_data = pedidos_data.withColumn('order_estimated_delivery_date', to_date(col('order_estimated_delivery_date')))

In [ ]:
#Agrupando por order_status
pedidos_data.groupBy('order_status').count().show()

+------------+-----+
|order_status|count|
+------------+-----+
|     shipped| 1107|
|    canceled|  625|
|    approved|    2|
|    invoiced|  314|
|     created|    5|
|   delivered|96478|
| unavailable|  609|
|  processing|  301|
+------------+-----+



In [ ]:
#Filtragem de pedidos para análise
pedidos_filtrados = pedidos_data.filter(col('order_status').isin(['delivered','shipped','approved']))
pedidos_filtrados.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_approved_at_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|                   2017-10-10|                   2017-10-18|            2017-10-02|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|              

In [ ]:
#Validação de volume
pedidos_contados = pedidos_df.count()
qtd_nulos = pedidos_df.filter(col('order_status').isNull()).count()
print(qtd_nulos)
print(pedidos_contados)

0
99441


In [ ]:
#Criação da métrica: tempo total de entrega
pedidos_filtrados = pedidos_filtrados.withColumn('total_days',datediff(col('order_delivered_customer_date'),col('order_approved_at_date')))
pedidos_filtrados.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+----------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_approved_at_date|total_days|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+----------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|                   2017-10-10|                   2017-10-18|            2017-10-02|         8|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27

In [ ]:
# Checagem de consistência da métrica `total_days`
pedidos_filtrados.filter(col('total_days').isNull()).count()

1131

In [ ]:
# Criação da métrica: atraso em relação ao prazo estimado
pedidos_filtrados = pedidos_filtrados.withColumn('days_delay',datediff(col('order_delivered_customer_date'),col('order_estimated_delivery_date')))
pedidos_filtrados.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+----------+----------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_approved_at_date|total_days|days_delay|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+----------+----------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|                   2017-10-10|                   2017-10-18|            2017-10-02|         8|        -8|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|

In [ ]:
#Classificação do status de entrega
pedidos_filtrados = pedidos_filtrados.withColumn('delivery_status',when(col('days_delay') > 0,'Atrasado').when(col('days_delay') < 0,'Adiantado').otherwise('No dia'))
pedidos_filtrados.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+----------+----------+---------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_approved_at_date|total_days|days_delay|delivery_status|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+----------+----------+---------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|                   2017-10-10|                   2017-10-18|            2017-10-02|         8|        -8|      Adi

In [ ]:
##Enriquecimento geográfico: mapeamento de estado para região
clientes_df = clientes_df.withColumn('region', when(col('customer_state').isin(['SP','MG','RJ','ES']),'Sudeste').when(col('customer_state').isin(['RS','SC','PR']),'Sul').\
                                     when(col('customer_state').isin(['GO','DF','MT','MS']),'Centro-Oeste').when(col('customer_state').isin(['AM','AC','AP','PA','RO','RR','TO']),'Norte').\
                                     when(col('customer_state').isin(['BA','SE','AL','PE','PB','RN','CE','PI']),'Nordeste').otherwise('Outros'))

In [ ]:
#Integração das tabelas (JOIN)
pedidos_clientes = pedidos_filtrados.join(
    clientes_df.select("customer_id", "customer_state","region"),
    on="customer_id",
    how="left"
)

In [ ]:
# Validação da tabela integrada
pedidos_clientes.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+----------+----------+---------------+--------------+------------+
|         customer_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_approved_at_date|total_days|days_delay|delivery_status|customer_state|      region|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------------+----------+----------+---------------+--------------+------------+
|9ef432eb625129730...|e481f51cbdc54678b...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|                   2017-10-10|

In [ ]:
#Preparação da base para análise regional
df_analise_regional = pedidos_clientes.withColumn(
    "dias_diferenca_entrega",
    datediff(
        col("order_delivered_customer_date"),
        col("order_estimated_delivery_date")
    )
).select(
    "order_id",
    "customer_id",
    "customer_state",
    "region",
    "delivery_status",
    "dias_diferenca_entrega",
    "order_purchase_timestamp",
    "order_estimated_delivery_date",
    "order_delivered_customer_date"
)

In [ ]:
#Agregação por região: criação dos KPIs
agg_regiao_entrega = df_analise_regional.groupBy("region").agg(
    count("*").alias("total_pedidos"),
    sum(when(col("delivery_status") == "Atrasado", 1).otherwise(0)).alias("pedidos_atrasados"),
    round(avg("dias_diferenca_entrega"), 2).alias("media_dias_diferenca")
)

agg_regiao_entrega = agg_regiao_entrega.withColumn(
    "percentual_atraso",
    round(col("pedidos_atrasados") / col("total_pedidos") * 100, 2)
)

In [ ]:
#Inspeção do resultado agregado
agg_regiao_entrega.show()

+------------+-------------+-----------------+--------------------+-----------------+
|      region|total_pedidos|pedidos_atrasados|media_dias_diferenca|percentual_atraso|
+------------+-------------+-----------------+--------------------+-----------------+
|    Nordeste|         8511|             1025|              -11.49|            12.04|
|      Outros|          734|              125|               -9.57|            17.03|
|         Sul|        13908|              815|              -13.08|             5.86|
|     Sudeste|        66907|             4048|              -11.56|             6.05|
|Centro-Oeste|         5702|              367|              -12.34|             6.44|
|       Norte|         1825|              154|              -15.61|             8.44|
+------------+-------------+-----------------+--------------------+-----------------+



In [ ]:
#Verificação de schema (tipos de dados)
agg_regiao_entrega.printSchema()

root
 |-- region: string (nullable = true)
 |-- total_pedidos: long (nullable = false)
 |-- pedidos_atrasados: long (nullable = true)
 |-- media_dias_diferenca: double (nullable = true)
 |-- percentual_atraso: double (nullable = true)



In [ ]:
#Exportação dos dados tratados
df_analise_regional.coalesce(1).write.mode("overwrite") \
    .option("header", True).csv("drive/MyDrive/E-commerce/df_analise_regional_csv")

agg_regiao_entrega.coalesce(1).write.mode("overwrite") \
    .option("header", True).csv("drive/MyDrive/E-commerce/agg_regiao_entrega_csv")